# AI Report William Forber Student ID: 22015706

### Importing and loading and printing the dataset:

In [1]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import pandas as pd
import tensorflow as tf
import keras
import keras_tuner as kt
from keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from baseline_model import base_model
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

K = 5
test_results = []
k_fold = KFold(n_splits=K, shuffle=True, random_state=42)

dataset = pd.read_csv('../Dataset/Synthetic_Data_For_Students.csv')
dataset.head(10)

2025-03-31 14:20:30.636906: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,SettlementValue,AccidentType,Injury_Prognosis,SpecialHealthExpenses,SpecialReduction,SpecialOverage,GeneralRest,SpecialAdditionalInjury,SpecialEarningsLoss,SpecialUsageLoss,...,Accident Date,Claim Date,Vehicle Age,Driver Age,Number of Passengers,Accident Description,Injury Description,Police Report Filed,Witness Present,Gender
0,520.0,Rear end,E. 5 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2023-11-10 11:22:24.508901,2024-06-11 11:22:24.508901,13.0,33.0,4.0,Side collision at an intersection.,Whiplash and minor bruises.,Yes,Yes,Male
1,870.0,Rear end,B. 2 months,0.0,0.0,0.0,520.0,0.0,0.0,90.0,...,2023-06-25 00:55:01.140228,2024-01-09 00:55:01.140228,4.0,45.0,2.0,Side collision at an intersection.,Minor cuts and scrapes.,Yes,Yes,Female
2,2140.0,Other side pulled out of side road,G. 7 months,0.0,0.0,0.0,1400.0,0.0,0.0,0.0,...,2020-02-23 17:43:47.805561,2020-03-01 17:43:47.805561,9.0,45.0,4.0,Lost control on a snowy road.,Whiplash and minor bruises.,Yes,No,Female
3,520.0,Rear end - Clt pushed into next vehicle,D. 4 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2021-10-02 04:36:32.118423,2021-10-13 04:36:32.118423,5.0,62.0,1.0,Side collision at an intersection.,Minor cuts and scrapes.,Yes,Yes,Female
4,260.0,Rear end,C. 3 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2023-04-02 05:13:07.117423,2023-04-14 05:13:07.117423,9.0,78.0,1.0,Lost control on a snowy road.,Concussion and bruised ribs.,Yes,Yes,Other
5,520.0,Rear end,F. 6 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2020-03-12 13:18:12.458491,2020-06-30 13:18:12.458491,10.0,55.0,1.0,Lost control on a snowy road.,Minor cuts and scrapes.,Yes,No,Other
6,1015.2,Rear end,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2023-04-08 08:24:57.899579,2023-08-19 08:24:57.899579,5.0,72.0,1.0,Hit a deer on the highway.,Whiplash and minor bruises.,No,No,Female
7,1032.0,Other side pulled out of side road,H. 8 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2021-10-14 11:00:13.682736,2022-02-19 11:00:13.682736,9.0,77.0,4.0,Side collision at an intersection.,Concussion and bruised ribs.,No,Yes,Female
8,808.2,Rear end,F. 6 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2020-09-09 15:07:57.455491,2021-02-04 15:07:57.455491,13.0,23.0,4.0,Swerved to avoid another vehicle.,Minor cuts and scrapes.,Yes,Yes,Female
9,500.0,Rear end,B. 2 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2021-09-01 12:37:18.207641,2021-10-25 12:37:18.207641,19.0,59.0,4.0,Rear-ended at a stoplight.,Minor cuts and scrapes.,Yes,Yes,Female


## Analysing the Dataset:

#### The dataset contains the settlement value which is the value I want to be predicting as well as the accident type, prognosis and the sub costs which all add to the settlement value, so we have various numerical, categorical data

## Pre-Processing the dataset:

##### I will divide the dataset into three categories, redundant labels, features that I don't need or will add complexity like the accident descripitions, dates and the sub costs to the settlement value, The other category will the categorical labels which I will use pandas get dummies to categoirse them, this includes Gender, acciedent type, whiplash etc, FInally the last category I have is the numerical labels being the settlement value, age, number of passengers etc.

In [2]:
redundant_labels = ['Accident Description', 'Claim Date', 'Accident Date',
                    'SpecialHealthExpenses', 'SpecialReduction', 'SpecialOverage', 'GeneralRest',
                    'SpecialAdditionalInjury', 'SpecialEarningsLoss', 'SpecialUsageLoss', 'SpecialMedications',
                    'SpecialAssetDamage', 'SpecialRehabilitation', 'SpecialFixes', 'GeneralFixed', 'GeneralUplift',
                    'SpecialLoanerVehicle', 'SpecialTripCosts', 'SpecialJourneyExpenses', 'SpecialTherapy']

category_labels = ['AccidentType', 'Exceptional_Circumstances', 'Minor_Psychological_Injury', 'Dominant injury',
                   'Whiplash', 'Vehicle Type', 'Weather Conditions',
                   'Police Report Filed', 'Witness Present', 'Gender', 'Injury Description']

numerical_labels = ['SettlementValue', 'Injury_Prognosis',
                    'Vehicle Age', 'Driver Age', 'Number of Passengers']

## Functions For Pre-Processing The Dataset:

#### Here I will define the functions and Pre-Process the dataset, So firstly I have a function which removes all rows that contain a null value to avoid mistraining my neural network, I Have a function which catagorises the data using pandas get dummies, finally for the numerical data I am scaling it with Robust scalar which reduces the impact of outliers, I'm returning the data as well as the scaler because I need to unscale the data when outputting the final mean abosulte error

In [3]:
def clean_dataset(data):
    data.dropna(inplace=True)
    data.drop(redundant_labels, axis=1, inplace=True)
    return data


def categorise_data(data, label):
    categories = pd.get_dummies(data[label])
    data.drop(label, axis=1, inplace=True)
    data = pd.concat([data, categories], axis=1)
    return data


def extract_months(prognosis):
    return int(''.join(filter(str.isdigit, prognosis)))


def scale_data(data):
    scaler = RobustScaler()
    data[numerical_labels] = scaler.fit_transform(data[numerical_labels])
    return data, scaler


def preprocess_data(data):
    data = clean_dataset(data)
    data['Injury_Prognosis'] = data['Injury_Prognosis'].apply(extract_months)
    data, scaler = scale_data(data)
    for label in category_labels:
        data = categorise_data(data, label)

    # outlier removal
    upper_limit = data['SettlementValue'].quantile(0.90)
    lower_limit = data['SettlementValue'].quantile(0.10)

    data['SettlementValue'] = data['SettlementValue'].clip(
        lower=lower_limit, upper=upper_limit)

    return data, scaler
